In [4]:
import math
import heapq
import numpy as np

class Node:
    def __init__(self, path, reduced_matrix, cost, vertex, level):
        self.path = path                
        self.reduced_matrix = reduced_matrix
        self.cost = cost                
        self.vertex = vertex                                                # trenutni grad
        self.level = level                                                  # nivo u stablu (koliko gradova je posjeceno)

    def __lt__(self, other):
        return self.cost < other.cost


def copy_matrix(matrix):
    return [row[:] for row in matrix]


# Redukcija matrice i racunanje cijene
def reduce_matrix(matrix):
    n = len(matrix)
    row_reduction = [math.inf] * n
    col_reduction = [math.inf] * n
    cost = 0

    # Redukcija po redovima
    for i in range(n):
        row_reduction[i] = min(matrix[i])
        if row_reduction[i] != math.inf and row_reduction[i] > 0:
            cost += row_reduction[i]
            for j in range(n):
                if matrix[i][j] != math.inf:
                    matrix[i][j] -= row_reduction[i]

    # Redukcija po kolonama
    for j in range(n):
        col_reduction[j] = min(matrix[i][j] for i in range(n))
        if col_reduction[j] != math.inf and col_reduction[j] > 0:
            cost += col_reduction[j]
            for i in range(n):
                if matrix[i][j] != math.inf:
                    matrix[i][j] -= col_reduction[j]

    return cost, matrix


# Generisanje novog čvora
def create_node(parent_matrix, path, level, i, j):
    n = len(parent_matrix)
    matrix = copy_matrix(parent_matrix)

    # iz grada A u grad B: u red A inf i u kolonu B inf
    for k in range(n):
        matrix[i][k] = math.inf
        matrix[k][j] = math.inf

    # ne mozemo se vratiti u pocetni cvor -> inf
    matrix[j][path[0]] = math.inf

    path = path[:]  
    path.append(j)

    # Redukcija matrice
    cost_reduction, reduced_matrix = reduce_matrix(matrix)

    return Node(path, reduced_matrix, cost_reduction, j, level)



def branch_and_bound_tsp(cost_matrix):
    n = len(cost_matrix)

    # Inicijalna redukcija
    cost, reduced_matrix = reduce_matrix(copy_matrix(cost_matrix))

    root = Node(path=[0], reduced_matrix=reduced_matrix, cost=cost, vertex=0, level=0)

    pq = []
    heapq.heappush(pq, root)

    best_cost = math.inf
    best_path = None

    while pq:
        # uzimam minimalni element u redu (onaj koji ima najmanju cijenu)
        min_node = heapq.heappop(pq)

        if min_node.level == n - 1:
            # vrati se na pocetak
            final_path = min_node.path + [0]
            final_cost = min_node.cost + cost_matrix[min_node.vertex][0]

            if final_cost < best_cost:
                best_cost = final_cost
                best_path = final_path
            continue

        # Grananje (za svaki sl grad)
        for j in range(n):
            if min_node.reduced_matrix[min_node.vertex][j] != math.inf:
                child = create_node(min_node.reduced_matrix, min_node.path,
                                    min_node.level + 1, min_node.vertex, j)
                child.cost = min_node.cost + cost_matrix[min_node.vertex][j] + child.cost

                if child.cost < best_cost:  
                    heapq.heappush(pq, child)

    return best_path, best_cost


if __name__ == "__main__":
    INF = math.inf
    cost_matrix = [ [INF, 12, 10, 19, 8, 14],
                    [12, INF, 3, 7, 2, 11],
                    [10, 3, INF, 6, 20, 4],
                    [19, 7, 6, INF, 13, 9], 
                    [8, 2, 20, 13, INF, 16], 
                    [14, 11, 4, 9, 16, INF] ]

    best_path, best_cost = branch_and_bound_tsp(cost_matrix)

    print("The best route for 1st matrix:", best_path)
    print("Min cost for 1st matrix:", best_cost)

    distance_matrix2 = [
        [0, 12, 10, 19, 8, 14, 11, 7, 15],
        [12, 0, 3, 7, 2, 11, 9, 6, 10],
        [10, 3, 0, 6, 20, 4, 8, 12, 5],
        [19, 7, 6, 0, 13, 9, 10, 15, 8],
        [8, 2, 20, 13, 0, 16, 7, 11, 14],
        [14, 11, 4, 9, 16, 0, 12, 8, 10],
        [11, 9, 8, 10, 7, 12, 0, 5, 13],
        [7, 6, 12, 15, 11, 8, 5, 0, 9],
        [15, 10, 5, 8, 14, 10, 13, 9, 0]
    ]

    best_path, best_cost = branch_and_bound_tsp(distance_matrix2)

    print("The best route for 2nd matrix:", best_path)
    print("Min cost for 2nd matrix:", best_cost)


The best route for 1st matrix: [0, 4, 1, 3, 5, 2, 0]
Min cost for 1st matrix: 77
The best route for 2nd matrix: [0, 7, 6, 3, 8, 5, 2, 1, 4, 0]
Min cost for 2nd matrix: 97
